In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.path.append('case_study')

import pandas as pd
import numpy as np

import joblib
from pathlib import Path
from sklearn import preprocessing

In [ ]:
# proj structure
# -input
#     --train.csv
#     --test.csv
# -src
#     --data_preprocess.py
#     --create_folds.py
#     --config.py
#     --model_dispatcher.py
#     --train.py
#     --inference.py
# -models
# -README.md
# -main.py

In [3]:
# src/data_preprocess.py

def data_preprocess():
    '''Preprocess data
    '''
    data_dir = Path('case_study/input')
    filepath = data_dir / 'DataScientist_CaseStudy_Dataset.xlsx'
    sheetnames = ['Description', 'Soc_Dem', 'Products_ActBalance', 'Inflow_Outflow', 'Sales_Revenues']
    
    sheet_id = 4
    df_sales_rev = pd.read_excel(filepath, sheet_name=sheetnames[sheet_id])
    
    sheet_id = 1
    df_user = pd.read_excel(filepath, sheet_name=sheetnames[sheet_id])
    
    sheet_id = 2
    df_acct = pd.read_excel(filepath, sheet_name=sheetnames[sheet_id])
    
    sheet_id = 3
    df_inout = pd.read_excel(filepath, sheet_name=sheetnames[sheet_id])
    
    # convert the column name to lowercase
    df_acct.columns = [col.lower() for col in df_acct.columns]
    df_user.columns = [col.lower() for col in df_user.columns]
    df_inout.columns = [col.lower() for col in df_inout.columns]
    df_sales_rev.columns = [col.lower() for col in df_sales_rev.columns]
    
    df = (
        df_user.merge(df_acct, on='client', how='inner')
        .merge(df_inout, on='client', how='outer')
        .merge(df_sales_rev, on='client', how='outer')
    )
    
    df_test = df[~df['client'].isin(df_sales_rev.client.unique())]
    filepath = data_dir / 'test.csv'
    df_test.to_csv(filepath, index=False)
    
    df = df[df['client'].isin(df_sales_rev.client.unique())]
    
    # process categorical variables
    df.loc[:, 'sex'] = df.sex.fillna('NONE')
    lbl_enc = preprocessing.LabelEncoder()
    df.loc[:, 'sex'] = lbl_enc.fit_transform(df.sex.values)
    
    encoder_fp = data_dir / 'sex_lbl_enc.bin'
    joblib.dump(lbl_enc, encoder_fp)
    
    for target_col in ['sale_mf', 'sale_cc', 'sale_cl']:
        df['target'] = np.where(df[target_col], 1, 0)
        print(f'Target: {target_col}')
        print(f'df.target.value_counts(): {df.target.value_counts()}')
        
        filepath = data_dir / f'{target_col}.csv'
        df.to_csv(filepath, index=False)


In [4]:
# src/create_folds.py

from sklearn import model_selection

def create_folds(fp_read, fp_save):
    '''Create folds for classfication problem
    '''
    df = pd.read_csv(fp_read)

    df['kfold'] = -1

    df = df.sample(frac=1).reset_index(drop=True)

    kf = model_selection.StratifiedKFold(n_splits=5)

    y = df.target.values
    
    for fold, (trn_, val_) in enumerate(kf.split(X=df, y=y)):
        df.loc[val_, 'kfold'] = fold

    df.to_csv(fp_save, index=False)


In [5]:
# src/config.py

FEAT_COLS = [
 'client',
 'sex',
 'age',
 'tenure',
 'count_ca',
 'count_sa',
 'count_mf',
 'count_ovd',
 'count_cc',
 'count_cl',
 'actbal_ca',
 'actbal_sa',
 'actbal_mf',
 'actbal_ovd',
 'actbal_cc',
 'actbal_cl',
 'volumecred',
 'volumecred_ca',
 'transactionscred',
 'transactionscred_ca',
 'volumedeb',
 'volumedeb_ca',
 'volumedebcash_card',
 'volumedebcashless_card',
 'volumedeb_paymentorder',
 'transactionsdeb',
 'transactionsdeb_ca',
 'transactionsdebcash_card',
 'transactionsdebcashless_card',
 'transactionsdeb_paymentorder']
print(f'number of feats to use: {len(FEAT_COLS)}')

DATA_DIR = Path('/case_study/input')
MODEL_DIR = Path('/case_study/models')

number of feats to use: 30


In [6]:
# src/model_dispatcher.py

from sklearn import tree
from sklearn import ensemble
# import lightgbm as lgbm

models = {
    'decision_tree_gini': tree.DecisionTreeClassifier(
        criterion='gini'
    ),
    'decision_tree_entropy': tree.DecisionTreeClassifier(
        criterion='entropy'
    ),
    'rf': ensemble.RandomForestClassifier(),
    # 'lgbm': lgbm.LGBMClassifier(
    #     boosting_type='gbdt', 
    #     num_leaves=31, 
    #     max_depth=-1, 
    #     learning_rate=0.1, 
    #     n_estimators=100, 
    #     subsample_for_bin=200000, 
    #     objective=None, 
    #     class_weight=None, 
    #     min_split_gain=0.0, 
    #     min_child_weight=0.001, 
    #     min_child_samples=20, 
    #     subsample=1.0, 
    #     subsample_freq=0, 
    #     colsample_bytree=1.0, 
    #     reg_alpha=0.0, 
    #     reg_lambda=0.0, 
    #     random_state=42, 
    #     n_jobs=None, 
    #     importance_type='split'
    # )
}

In [7]:
# src/train.py

import joblib
import pandas as pd
from sklearn import metrics

def run(fold, model_name, target_col):
    '''Train model
    '''
    fp = DATA_DIR / f'{target_col}_train_folds.csv'
    df = pd.read_csv(fp)

    df_train = df[df.kfold != fold].reset_index(drop=True)

    df_valid = df[df.kfold == fold].reset_index(drop=True)

    x_train = df_train[FEAT_COLS].values
    y_train = df_train.target.values

    x_valid = df_valid[FEAT_COLS].values
    y_valid = df_valid.target.values

    clf = models[model_name]

    clf.fit(x_train, y_train)

    preds = clf.predict(x_valid)

    auc = metrics.roc_auc_score(y_valid, preds)
    print(f'Fold={fold}, ROC AUC={auc}')

    model_fp = MODEL_DIR / f'{target_col}_{model_name}_{fold}.bin'
    joblib.dump(clf, model_fp)

In [8]:
# src/inference.py

def get_inference(target_col):
    '''Get inference
    '''
    filepath = DATA_DIR / 'test.csv'
    df_test = pd.read_csv(filepath)
    
    encoder_fp = DATA_DIR / 'sex_lbl_enc.bin'
    lbl_enc = joblib.load(encoder_fp)
    
    df_test.loc[:, 'sex'] = df_test['sex'].fillna('NONE')
    df_test.loc[:, 'sex'] = lbl_enc.transform(df_test['sex'].values)
    
    x_test = df_test[FEAT_COLS].values
    model_name = 'rf'
    pred_lst = []
    probs_lst = []
    for fold in range(5):
        model_fp = MODEL_DIR / f'{target_col}_{model_name}_{fold}.bin'
        model = joblib.load(model_fp)
        
        probs = model.predict_proba(x_test)
        
        preds = np.argmax(probs, axis=1, keepdims=True)
        probs = np.take_along_axis(probs, preds, axis=1)
        
        probs_lst.append(probs)
        pred_lst.append(preds)
        
    thresh = 0.5
    preds = (np.mean(np.array(pred_lst), axis=0) > thresh).astype(int)
    probs = np.mean(np.array(probs_lst), axis=0)
    
    df_pred = pd.DataFrame(
        data={
            'client': df_test.client.values,
            f'preds_{target_col}': preds.squeeze(),
            f'probs_{target_col}': probs.squeeze()
        }
    )
    pred_fp = DATA_DIR / f'pred_{target_col}.csv'
    df_pred.to_csv(pred_fp, index=False)

    return df_pred


In [9]:

print(f'\n======Step 1. Data Preprocessing, create train and test dataset======')
data_dir = Path('case_study/input')
data_preprocess()

model_name = 'rf'
for target_col in ['sale_mf', 'sale_cc', 'sale_cl']:
    fp_read = data_dir / f'{target_col}.csv'
    fp_save = data_dir / f'{target_col}_train_folds.csv'
    print(f'\n======Step 2. Create cross validation for {target_col}======')
    create_folds(fp_read, fp_save)

    print(f'\n======Step 3. Run {model_name} for target {target_col}======')
    for fold in range(5):
        run(fold=fold, model_name=model_name, target_col=target_col)

print(f'\n======Step 4. Get inference======')
df_pred_sale_mf = get_inference(target_col='sale_mf')
df_pred_sale_cc = get_inference(target_col='sale_cc')
df_pred_sale_cl = get_inference(target_col='sale_cl')


# combine the preds, get the top clients
print(f'\n======Step 5. Select top clients======')
df_pred = (
    df_pred_sale_mf.merge(df_pred_sale_cc, on='client', how='inner')
    .merge(df_pred_sale_cl, on='client', how='inner')
)

df_pred['offer_cnt'] = df_pred['preds_sale_mf'] + df_pred['preds_sale_cc'] + df_pred['preds_sale_cl']

df_pred['final_offer'] = np.where(
    df_pred['offer_cnt'] > 0,
    df_pred[['probs_sale_mf', 'probs_sale_cc', 'probs_sale_cl']].idxmax(axis=1),
    'No offer'
)

df_pred['sale_mf'] = np.where(
    df_pred['final_offer'].str.contains('sale_mf'), 1, 0
)
df_pred['sale_cc'] = np.where(
    df_pred['final_offer'].str.contains('sale_cc'), 1, 0
)
df_pred['sale_cl'] = np.where(
    df_pred['final_offer'].str.contains('sale_cl'), 1, 0
)

fp_save = DATA_DIR / 'pred_sale.csv'
df_top = df_pred[df_pred['offer_cnt'] > 0][['client', 'sale_mf', 'sale_cc', 'sale_cl']]
df_top.to_csv(fp_save, index=False)

print(f'\n======Step 6. Get final revenue======')
fp_read = data_dir / f'{target_col}.csv'
df = pd.read_csv(fp_read)

mean_mf = df[df['sale_mf'] == 1].revenue_mf.mean()
mean_cc = df[df['sale_cc'] == 1].revenue_cc.mean()
mean_cl = df[df['sale_cl'] == 1].revenue_cl.mean()
print(
    f'\nmean_mf: {mean_mf:.2f}'
    f'\nmean_cc: {mean_cc:.2f}'
    f'\nmean_cl: {mean_cl:.2f}'
)

final_revenue = df_top['sale_mf'].sum() * mean_mf + df_top['sale_cc'].sum() * mean_cc + df_top['sale_cl'].sum() * mean_cl
print(f'final revenue: {final_revenue:.2f}')


======Step 1. Data Preprocessing, create train and test dataset======
Target: sale_mf
df.target.value_counts(): target
0    776
1    193
Name: count, dtype: int64
Target: sale_cc
df.target.value_counts(): target
0    727
1    242
Name: count, dtype: int64
Target: sale_cl
df.target.value_counts(): target
0    679
1    290
Name: count, dtype: int64

======Step 2. Create cross validation for sale_mf======

======Step 3. Run rf for target sale_mf======
Fold=0, ROC AUC=0.5330634278002699
Fold=1, ROC AUC=0.5511993382961126
Fold=2, ROC AUC=0.5095947063688998
Fold=3, ROC AUC=0.5256410256410257
Fold=4, ROC AUC=0.5362478777589135

======Step 2. Create cross validation for sale_cc======

======Step 3. Run rf for target sale_cc======
Fold=0, ROC AUC=0.562785388127854
Fold=1, ROC AUC=0.593892694063927
Fold=2, ROC AUC=0.5679802955665024
Fold=3, ROC AUC=0.5711470795214638
Fold=4, ROC AUC=0.5799568965517241

======Step 2. Create cross validation for sale_cl======

======Step 3. Run rf for target sale